In [2]:
!nvidia-smi

Tue Nov 12 11:02:44 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:07:00.0 Off |                   On |
| N/A   43C    P0            196W /  400W |                  N/A   |     N/A      Default |
|                                         |                        |              Enabled |
+-----------------------------------------+-----

In [19]:
import os

import torch
from torch import cuda
from torchvision.models import resnet50
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm

In [4]:
torch_device_target = 'cpu'
if cuda.is_available():
    print('cuda available: ', cuda.is_available())
    print('cuda device count: ', cuda.device_count())
    print('cuda current device: ', cuda.current_device())
    print('cuda device: ', cuda.device(cuda.current_device()))
    print('cuda device name: ', cuda.get_device_name(cuda.current_device()))
    torch_device_target = 'cuda'
else:
    print('running with cuda disabled')
    
device = torch.device(torch_device_target)

cuda available:  True
cuda device count:  1
cuda current device:  0
cuda device:  <torch.cuda.device object at 0x7fe41f233ac0>
cuda device name:  NVIDIA A100-SXM4-40GB MIG 1g.10gb


# Data Preparation

In [5]:
ROOT_METADATA_PATH = '/workspace/datasets/train-series-splitted-metadata/'
TRAIN_PATH = os.path.join(ROOT_METADATA_PATH, 'train800.csv')
TEST_PATH = os.path.join(ROOT_METADATA_PATH, 'test100.csv')
VALID_PATH= os.path.join(ROOT_METADATA_PATH, 'valid100.csv')

In [6]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
valid_df = pd.read_csv(VALID_PATH)

ich_labels = ['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']


In [7]:
train_df

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,PatientID,StudyInstanceUID,SeriesInstanceUID,ImagePositionPatient,ImageId,ImagePositionSpan
0,ID_00008ce3c,0,0,0,0,0,0,ID_ce8a3cd2,ID_974735bf79,ID_3780d48b28,"['-125', '-83.0468112', '175.995344']",ID_00008ce3c,175.995344
1,ID_000259ccf,0,0,0,0,0,0,ID_ee4d05e4,ID_127d3011a5,ID_6254afb327,"['-125.000', '-121.815', '187.706']",ID_000259ccf,187.706000
2,ID_000457398,0,0,0,0,0,0,ID_34fffd88,ID_72c127d025,ID_5ad9e0e469,"['-126.437378', '-126.437378', '62.500000']",ID_000457398,62.500000
3,ID_0005de569,0,0,0,0,0,0,ID_7ab6d7df,ID_b4d83b111f,ID_16801bed0a,"['-125.000', '-97.286', '-1.590']",ID_0005de569,-1.590000
4,ID_0006441d0,0,0,0,0,0,0,ID_4681a956,ID_5b133aa2c0,ID_a9000bdf3b,"['-125.000000', '-122.597977', '177.810806']",ID_0006441d0,177.810806
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27166,ID_fffc1b4a8,0,0,0,0,0,0,ID_f5ac09ac,ID_906e006b9f,ID_2f58ba6ff8,"['-125', '6', '112.799988']",ID_fffc1b4a8,112.799988
27167,ID_fffdd6240,0,0,0,0,0,0,ID_2378f0b7,ID_831a4f4f7d,ID_d91562c643,"['-125.000', '-102.398', '156.077']",ID_fffdd6240,156.077000
27168,ID_fffecf238,0,0,0,0,0,0,ID_33e26eaf,ID_106dc48992,ID_7b296147d3,"['-125', '-10', '207']",ID_fffecf238,207.000000
27169,ID_ffff922b9,1,0,0,1,0,0,ID_5964c5e5,ID_b47ca0ad05,ID_6d2a9b2810,"['-126.408875', '-126.408875', '-235.611511']",ID_ffff922b9,-235.611511


In [8]:
# train_df.iloc[700][ich_labels].values.astype(np.int32)

In [9]:
class ICHSinogramDataset(Dataset):
    def __init__(self, metadata, image_dir, transform=None):
      self.metadata = metadata
      self.image_dir = image_dir
      self.transform = transform
      
    def __len__(self):
      return len(self.metadata)
    
    def __getitem__(self, idx):
      row = self.metadata.iloc[idx]
      img_path =  os.path.join(self.image_dir, f"{row['Image']}.npy")
      img_arr = np.load(img_path)
      img_arr = np.squeeze(img_arr)
      # img_arr = np.clip(img_arr, 0, 1)
      img_arr = np.stack([img_arr.copy() for _ in range(3)], axis=-1)
      if self.transform:
        img_arr = self.transform(img_arr)
          
      labels = torch.from_numpy(row[ich_labels].values.astype(np.int32))
      
      return img_arr, labels

In [10]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
img_dir = '/workspace/datasets/processed/sinograms'
train_dataset = ICHSinogramDataset(metadata=train_df, image_dir=img_dir, transform=transform)
test_dataset = ICHSinogramDataset(metadata=test_df, image_dir=img_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=4)



In [12]:
# s = np.load(os.path.join(img_dir,f"{train_df['Image'][0]}.npy"))
# s = np.squeeze(s)
# s = np.clip(s,0,1)
# s = np.stack((s,)*3, axis=-1)
# s.shape

In [13]:
images, labels = next(iter(train_loader))
print("Batch of images: ", images.shape)
print("Batch of labels: ", labels.shape)


Batch of images:  torch.Size([32, 3, 224, 224])
Batch of labels:  torch.Size([32, 6])


In [14]:
class ICHResNet50(nn.Module):
    def __init__(self, num_classes=6):
        super(ICHResNet50, self).__init__()
        self.resnet = resnet50(weights=None)
        self.in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(self.in_features, num_classes)
      
    def forward(self, x):
        return self.resnet(x)

In [15]:
model = ICHResNet50(num_classes=6).cuda()
eta = 0.00005
num_epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=eta)

epoch_train_loss = []

In [20]:
for epoch in range(num_epochs):
    train_losses = []
    model.train()
    
    # Wrap train_loader with tqdm
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
        optimizer.zero_grad()
        
        # Move data and labels to GPU
        data = images.float().cuda()
        labels = labels.float().cuda()
        
        # Forward pass
        y_hat = model(data)
        error = nn.BCEWithLogitsLoss()
        loss = error(y_hat, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
    # Calculate and print epoch loss
    epoch_loss = np.mean(train_losses)
    epoch_train_loss.append(epoch_loss)
    print(f'Train Epoch: {epoch+1}/{num_epochs}\tTrain Loss: {epoch_loss:.6f}')

Train Epoch: 1/10	Train Loss: 0.305005


Train Epoch: 2/10	Train Loss: 0.285079


Train Epoch: 3/10	Train Loss: 0.276053


Train Epoch: 4/10	Train Loss: 0.266524


Train Epoch: 5/10	Train Loss: 0.255053


Train Epoch: 6/10	Train Loss: 0.241418


Train Epoch: 7/10	Train Loss: 0.220991


Train Epoch: 8/10	Train Loss: 0.192699


Train Epoch: 9/10	Train Loss: 0.161671


Train Epoch: 10/10	Train Loss: 0.130559
